In [1]:
#!pip install pandas
#!pip install nltk
# !pip install langdetect
# !pip install scikit-learn gensim
import pandas as pd


In [2]:
df = pd.read_csv('Historical Lead Records.csv', encoding='latin1')

df.head()

,Title,Job Role,Job Function,Job Level
0,Manager-Cybersecurity,Information Security,IT,Manager
1,"Manager, Information Security",Information Security,IT,Manager
2,User Experience Analyst,Development,Engineering,Contributor
3,Network Specialist,Networking,IT,Contributor
4,Director of Privacy and Compliance,Information Security,IT,Director


In [3]:
#make title column lowercase
df['Title'] = df['Title'].str.lower()

df.shape[0]

865671

In [4]:
#data relableing to most frequent F/R/L
def most_frequent(x):
    if not x.empty and len(x.value_counts()) > 0:
        return x.value_counts().index[0]
    else:
        return None

grouped = df.groupby('Title').agg(most_frequent)
grouped.reset_index(inplace=True)

merged_df = pd.merge(df, grouped, on='Title', suffixes=('', '_most_frequent'))

merged_df.drop(['Job Role', 'Job Function', 'Job Level'], axis=1, inplace=True)

merged_df.rename(columns={'Job Role_most_frequent': 'Job Role', 'Job Function_most_frequent': 'Job Function', 'Job Level_most_frequent': 'Job Level'}, inplace=True)


In [5]:
na_counts = df.isna().sum()
print(na_counts)

Title           12690
Job Role         4502
Job Function     5083
Job Level       24546
dtype: int64


In [6]:
#1616 rows are completely empty
before_dropping = df.shape[0]
empty_rows= df.isna().all(axis=1).sum()
df = df.dropna(how='all')
after_dropping = df.shape[0]


print (before_dropping)
print (empty_rows)
print (after_dropping)

865671
1616
864055


In [7]:
question_mark = df['Title'].str.contains('\?').sum()
question_mark

148

In [8]:
# #replacing all question marks with empty strings
df['Title'] = df['Title'].str.replace('?', '')
question_mark1 = df['Title'].str.contains('\?').sum()
question_mark1

0

In [9]:
df = df.dropna(subset=['Title'])
df.shape[0]

852981

In [12]:
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException
from joblib import Parallel, delayed
import numpy as np

def is_english(text):
    try:
        return detect(str(text)) == 'en'
    except LangDetectException:
        return False

def filter_english_titles(df):
    return df[df['Title'].apply(is_english)]

num_jobs = 4
chunks = np.array_split(df, num_jobs)
df_filtered_chunks = Parallel(n_jobs=num_jobs)(
    delayed(filter_english_titles)(chunk) for chunk in chunks
)
df_filtered = pd.concat(df_filtered_chunks)



In [13]:
# since the cleaning steps above take a while to run, output df_filtered to csv
df_filtered.to_csv('df_filtered.csv', index=False)

In [14]:
df_filtered.shape[0]

550384

In [15]:
df_filtered.head()

,Title,Job Role,Job Function,Job Level
0,manager-cybersecurity,Information Security,IT,Manager
1,"manager, information security",Information Security,IT,Manager
2,user experience analyst,Development,Engineering,Contributor
3,network specialist,Networking,IT,Contributor
4,director of privacy and compliance,Information Security,IT,Director


In [16]:
def replace_role(row):
    if pd.isnull(row['Job Function']): 
        return 'N/A'
    elif 'IT' not in row['Job Function']:
        return 'N/A'
    else:
        return row['Job Role']

df_filtered['Job Role'] = df_filtered.apply(replace_role, axis=1)

In [17]:
df_filtered['Job Function'] = df_filtered['Job Function'].fillna('')

non_it = df_filtered[~df_filtered['Job Function'].str.contains('IT')]
non_it.head()

,Title,Job Role,Job Function,Job Level
2,user experience analyst,N/A,Engineering,Contributor
14,"branch manager, purchasing",N/A,Purchasing,Manager
18,legislative affairs director,N/A,Legal,Director
25,associate director of software development,N/A,Engineering,Director
28,turning vision into reality,N/A,Engineering,Unknown


In [22]:
df = df_filtered.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [23]:
job_levels = df['Job Level'].value_counts()
job_functions = df['Job Function'].value_counts()

print(job_levels.head())
print(job_functions.head())

Job Level
Contributor    185866
Manager        115397
Director       109276
Executive       70486
C-Level         56543
Name: count, dtype: int64
Job Function
IT             444323
Engineering     45012
Sales           13092
Operations       8390
Management       7453
Name: count, dtype: int64


In [24]:
# Remove any stop words from the titles
from gensim.parsing.preprocessing import remove_stopwords
df['Title'] = df['Title'].apply(remove_stopwords)

In [25]:
import gensim 
#Some more preprocessing that will remove punctuations. 
tokenised_titles = df['Title'].apply(gensim.utils.simple_preprocess)

In [26]:
tokenised_titles.head()

0            [manager, cybersecurity]
1    [manager, information, security]
2         [user, experience, analyst]
3               [network, specialist]
4     [director, privacy, compliance]
Name: Title, dtype: object

In [27]:
#add the tokens as a column in df
df.insert(1, 'Title_Tokens', tokenised_titles)
df.head()

,Title,Title_Tokens,Job Role,Job Function,Job Level
0,manager-cybersecurity,"[manager, cybersecurity]",Information Security,IT,Manager
1,"manager, information security","[manager, information, security]",Information Security,IT,Manager
2,user experience analyst,"[user, experience, analyst]",N/A,Engineering,Contributor
3,network specialist,"[network, specialist]",Networking,IT,Contributor
4,director privacy compliance,"[director, privacy, compliance]",Information Security,IT,Director


In [28]:
#initialize and train model
model = gensim.models.Word2Vec(window=2,min_count=1, workers=4)
model.build_vocab(df['Title_Tokens'])#this is a required step before training the model 
model.train(df['Title_Tokens'], total_examples=model.corpus_count, epochs=model.epochs) #default vector_size = 100


(4458100, 9898370)

In [29]:
model.wv.most_similar("manager")

[('director', 0.7009299993515015),
 ('leader', 0.6988219022750854),
 ('mgr', 0.671646773815155),
 ('manger', 0.6508705019950867),
 ('supervisor', 0.6294075846672058),
 ('lead', 0.620506763458252),
 ('analyst', 0.603598952293396),
 ('head', 0.6028900146484375),
 ('specialist', 0.5945497155189514),
 ('coordinator', 0.5887508988380432)]

In [30]:
model.wv.most_similar("analyst")

[('specialist', 0.754058301448822),
 ('analysts', 0.7430660724639893),
 ('technician', 0.6369121670722961),
 ('intern', 0.621850311756134),
 ('manger', 0.6095274686813354),
 ('manager', 0.603598952293396),
 ('engineer', 0.6031732559204102),
 ('cloudselect', 0.5979779958724976),
 ('enginner', 0.5970101952552795),
 ('spec', 0.5925499796867371)]

In [31]:
model.wv.most_similar("network")

[('noc', 0.6353093981742859),
 ('networks', 0.5984349250793457),
 ('installation', 0.5947428345680237),
 ('carrier', 0.5877697467803955),
 ('centralized', 0.5877643823623657),
 ('broadcast', 0.5738204121589661),
 ('video', 0.5697857141494751),
 ('eoc', 0.5618852376937866),
 ('omo', 0.5563306212425232),
 ('networking', 0.5404847860336304)]

In [32]:
import numpy as np

#below function creates a vector for a title by adding the vector of each word in it
def get_title_vec(title_tokens):
    # create a list of vectors of all the tokens in a title
    vectors = [ model.wv[token] for token in title_tokens]
    #sum all the vectors in the list
    return np.sum(vectors, axis=0)

title_vecs = df['Title_Tokens'].map(get_title_vec)
#Add title vecs as a column in df 
df.insert(2, 'Title_Vec', title_vecs)
df.head()

,Title,Title_Tokens,Title_Vec,Job Role,Job Function,Job Level
0,manager-cybersecurity,"[manager, cybersecurity]","[-0.45688772, 1.648301, -0.4126865, -1.1483161...",Information Security,IT,Manager
1,"manager, information security","[manager, information, security]","[-0.58403826, 3.7550025, -0.2716064, -1.603669...",Information Security,IT,Manager
2,user experience analyst,"[user, experience, analyst]","[-1.0252491, 2.248923, -0.5732887, -3.41615, 0...",N/A,Engineering,Contributor
3,network specialist,"[network, specialist]","[-0.5637119, 1.987749, -0.42509258, 0.640834, ...",Networking,IT,Contributor
4,director privacy compliance,"[director, privacy, compliance]","[0.3347448, 2.9563966, -0.5382847, 0.24741949,...",Information Security,IT,Director


In [35]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ltobin7\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [40]:
# First set up DTM
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
stops = stopwords.words('english')

# Vectorize titles 
vec = CountVectorizer(token_pattern = r'\b[a-zA-Z_]{3,}[a-zA-Z]*\b', # DO NOT REMOVE THIS LINE!!!!!!!!!
                      max_df=0.5,
                      lowercase=True, 
                      stop_words=list(stops), 
                      max_features=1000, ngram_range=(1,2)) 
dtm = vec.fit_transform(df['Title'])

In [41]:
# Create training and validation samples
from sklearn.model_selection import train_test_split

trainX,validX,trainy,validy = train_test_split(dtm, df['Job Function'],train_size=0.80,random_state=123)

In [ ]:
# Start with Bayes Naive Classifier to determine Job Function 

In [ ]:
#  From here - create two additional Naive Bayes Classifiers
# (1) Tune a Naive Bayes Classifier to identify Job Role for IT Function
# (2) Tune Naive Bayes Classifier to identify Job Level for IT and non-IT (but not non-ICP)


# Also worth checking performance (if time) - is it better if there is just one Naive Bayes Classifier for all job title
# to determine function, role, level? 

In [ ]:
# Once final model is decided, need to create code to input CSV file with Record ID, Job Title
# Output will be Record ID, Job Title, Job Function, Job Role, Job Level 